In [205]:
import pandas as pd
import os
import datetime as dt
from typing import List

In [ ]:
# ================================================================
# Utility Functions
# ================================================================
def get_last_n_years(n: int = 4) -> List[int]:
    """
    Get a list of the most recent N years including the current year.

    Returns:
        List[int]: List of years in descending order (e.g., [2025, 2024, 2023, 2022]).
    """
    current_year = dt.date.today().year
    return list(range(current_year, current_year - n, -1))

def get_last_n_year_data(folder='../partitioned_output', years: int = 4) -> pd.DataFrame:
    """
    Load parquet files for the last N years from a partitioned folder structure.

    The folder is assumed to have year-based subdirectories (e.g., "year=2023").

    Args:
        folder (str, optional): Root folder path. Defaults to '../partitioned_output'.
        years (int, optional): Number of recent years to include. Defaults to 4.

    Returns:
        pd.DataFrame: Combined DataFrame with all rows from the relevant parquet files.
    """    # Get valid years
    valid_years = set(get_last_n_years(years))
    # Initialize list to hold DataFrames
    df_files = []

    # Walk through the directory
    for dirpath, dirnames, filenames in os.walk(folder):
        for dirname in dirnames:
            # Check if the directory name contains a year
            dir_year = int(dirname.split('=')[-1])
            if dir_year in valid_years:
                # If it does, read all Parquet files in that directory
                dir_full_path = os.path.join(dirpath, dirname)
                df_temp = pd.DataFrame()
                for file in os.listdir(dir_full_path):
                    # Read the Parquet file and append it to the list
                    if file.endswith('.parquet'):
                        file_path = os.path.join(dir_full_path, file)
                        df = pd.read_parquet(file_path)
                        df_temp = pd.concat([df_temp, df], axis=0)
                # Append the DataFrame from this directory to the list
                df_files.append(df_temp)
    # Concatenate all DataFrames into a single DataFrame
    df = pd.concat(df_files, axis=0)
    # Reset the index
    df.reset_index(drop=True, inplace=True)
    # Return the final DataFrame
    return df

In [ ]:
def get_period_boundaries(today=None):
    """
    Compute time period boundaries (R12, Prior R12, R6, Prior R6, YTD, Prior YTD).

    Args:
        today (pd.Timestamp, optional): Reference date. Defaults to today.

    Returns:
        dict: Mapping of period names to (start_date, end_date) as pd.Timestamp.
    """
    today = (today or pd.Timestamp.today()).normalize()
    # Define period start dates
    ytd_start = pd.Timestamp(year=today.year, month=1, day=1)
    start_r12 = today - pd.Timedelta(days=365)
    start_r6 = today - pd.DateOffset(months=6)
    prior_r12_start = start_r12 - pd.DateOffset(months=12)
    prior_r6_start = start_r6 - pd.DateOffset(months=6)
    # Corrected prior YTD
    prior_ytd_start = ytd_start - pd.DateOffset(years=1)
    prior_ytd_end = ytd_start - pd.Timedelta(days=1)

    periods = {
        "R12": (start_r12, today),
        "Prior R12": (prior_r12_start, today - pd.DateOffset(months=12)),
        "R6": (start_r6, today),
        "Prior R6": (prior_r6_start, today - pd.DateOffset(months=6)),
        "YTD": (ytd_start, today),
        "Prior YTD": (prior_ytd_start, prior_ytd_end)
    }
    return periods
def slice_data(df, start=None, end=None, report_type=None):
    """
    Slice the dataset for a specific date range and add metadata columns.

    Args:
        df (pd.DataFrame): Input DataFrame with at least a 'Date' column.
        start (pd.Timestamp): Start date of the slice.
        end (pd.Timestamp): End date of the slice.
        report_type (str): Label for the slice (e.g., "R12", "YTD").

    Returns:
        pd.DataFrame: Filtered slice with Report Type, Start Date, End Date columns.
    """
    # Slice the DataFrame for the given date range
    df_slice = df[(df['Date'] >= start) & (df['Date'] <= end)].copy()
    # Add metadata columns
    df_slice["Report Type"] = report_type
    df_slice["Start Date"] = start
    df_slice["End Date"] = end
    # Append to the list
    if df_slice.empty:
        return
    else:
        return df_slice
def categorize_crime(fbi_code):
    """
    Categorize a crime record into Violent, Property, or Other based on FBI code.

    Args:
        fbi_code (str): FBI crime classification code.

    Returns:
        str: "Violent", "Property", or "Other"
    """
    VIOLENT_CODES = {"01A", "02", "03", "04A", "04B", "05"}
    PROPERTY_CODES = {"06", "07", "09", "10"}


    if fbi_code in VIOLENT_CODES:
        return "Violent"
    elif fbi_code in PROPERTY_CODES:
        return "Property"
    else:
        return "Other" 
def summary_df(df):
    """
    Aggregate crime dataset into summary statistics by report type and date range.

    Returns:
        DataFrame with counts, unique categories, arrest/domestic/violent/property rates.
    """
    # Categorization and Flags
    df['Crime Category'] = df['FBI Code'].apply(categorize_crime)
    df['Violent'] = (df['Crime Category'] == "Violent").astype(int)
    df['Property'] = (df['Crime Category'] == "Property").astype(int)
    # Convert to date only
    df['Start Date'] = pd.to_datetime(df['Start Date']).dt.date
    df['End Date'] = pd.to_datetime(df['End Date']).dt.date

    df_agg = df\
        .groupby(['Report Type','Start Date','End Date'])\
        .agg({
            'Case Number': 'count',
            'Primary Type': pd.Series.nunique,
            'Arrest': 'sum',
            'Domestic': 'sum',
            'Violent': 'sum',
            'Property': 'sum',
            'Beat': pd.Series.nunique,
            'Ward': pd.Series.nunique,
            'District': pd.Series.nunique
        })\
        .rename(columns={
            'Case Number': 'Total Cases',
            'Primary Type': 'Unique Crime Types',
            'Arrest': 'Total Arrests',
            'Domestic': 'Total Domestic Cases',
            'Violent': 'Total Violent Cases',
            'Property': 'Total Property Cases',
            'Beat': 'Unique Beats',
            'Ward': 'Unique Wards',
            'District': 'Unique Districts'
        })\
        .reset_index()

    df_agg['Arrest Rate'] = (df_agg['Total Arrests'] / df_agg['Total Cases'] * 100).round(2)
    df_agg['Domestic Rate'] = (df_agg['Total Domestic Cases'] / df_agg['Total Cases'] * 100).round(2)
    df_agg['Violent Rate'] = (df_agg['Total Violent Cases'] / df_agg['Total Cases'] * 100).round(2)
    df_agg['Property Rate'] = (df_agg['Total Property Cases'] / df_agg['Total Cases'] * 100).round(2)

    return df_agg
def add_growth(df_agg):
    """
    Add growth/change metrics by explicitly aligning current vs prior periods.
    """

    # Define mapping of "current" to "prior" period names
    period_pairs = {
        "R12": "Prior R12",
        "R6": "Prior R6",
        "YTD": "Prior YTD"
    }

    df_with_growth = []

    for current, prior in period_pairs.items():
        # Separate current and prior subsets
        df_current = df_agg[df_agg['Report Type'] == current].copy()
        df_prior = df_agg[df_agg['Report Type'] == prior].copy()

        # Align on Start Date (or End Date if you prefer)
        merged = pd.merge(
            df_current,
            df_prior,
            left_on=['Start Date'],
            right_on=['End Date'],
            suffixes=('', '_prior')
        )

        # Dynamic growth metrics
        for col in ["Arrest Rate", "Domestic Rate", "Violent Rate", "Property Rate"]:
            merged[f"{col} Change"] = merged[col] - merged[f"{col}_prior"]

        # Case growth %
        merged["Case Growth %"] = (
            (merged['Total Cases'] - merged['Total Cases_prior'])
            / merged['Total Cases_prior'] * 100
        ).round(2)

        df_with_growth.append(merged)

    # Concatenate results for all period types
    return pd.concat(df_with_growth, ignore_index=True)

In [207]:
df = get_last_n_year_data(years=5)
# Ensure Date is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# min and max date
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
# show dataframe shape
print(f"Dataframe shape: {df.shape}")
# show columns
print(f"Columns: {df.columns.tolist()}")
# show data types
print(f"Data types:\n{df.dtypes}")

Date range: 2021-01-01 00:00:00 to 2025-09-19 00:00:00
Dataframe shape: (1141526, 22)
Columns: ['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location']
Data types:
ID                               int64
Case Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                            object
District                        object
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   floa

In [208]:
# show first 3 rows
display(df.head(3))


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12375131,JE241940,2021-05-24 10:00:00,070XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,SCHOOL - PRIVATE GROUNDS,False,False,...,6.0,69.0,08B,1178378.0,1858441.0,2021,2021-05-31 15:41:02,41.766869,-87.621729,"(41.766869275, -87.62172935)"
1,12375284,JE240916,2021-05-24 10:00:00,016XX W CERMAK RD,0430,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,SIDEWALK,False,False,...,25.0,31.0,04B,1165877.0,1889479.0,2021,2021-05-31 15:41:02,41.852316,-87.666668,"(41.852315746, -87.666667947)"
2,12379216,JE247002,2021-05-24 10:00:00,010XX N STATE ST,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,...,2.0,8.0,11,1176175.0,1907421.0,2021,2021-06-01 15:39:54,41.901324,-87.628331,"(41.90132405, -87.628330712)"


In [210]:
# Get a range of end of month dates for the last 4 years
end_of_months = pd.date_range(end=pd.Timestamp.today(), periods=36, freq='M')
# Store list of period dates
list_period_boundries = []
# Store Slice Period DataFrames
list_period_dfs = []
# Loop through each end of month date, slice the data, compute summary stats, and aggregate
for period_end in end_of_months:
    # get period boundaries for the given period_end
    list_period_boundries.append(get_period_boundaries(period_end))

for row in list_period_boundries:
    r12 = row["R12"]
    ytd = row["YTD"]
    prior_r12 = row["Prior R12"]
    prior_ytd = row["Prior YTD"]
    # Slice data for R12 and YTD periods
    period_dfs = slice_data(df, start = r12[0], end = r12[1], report_type="R12")
    # aggregate summary stats
    list_period_dfs.append(summary_df(period_dfs))
    # slice data for YTD and Prior periods
    period_dfs = slice_data(df, start = ytd[0], end = ytd[1], report_type="YTD")
    list_period_dfs.append(summary_df(period_dfs))
    period_dfs = slice_data(df, start = prior_r12[0], end = prior_r12[1], report_type="Prior R12")
    list_period_dfs.append(summary_df(period_dfs))
    period_dfs = slice_data(df, start = prior_ytd[0], end = prior_ytd[1], report_type="Prior YTD")
    list_period_dfs.append(summary_df(period_dfs))
# Combine all period DataFrames into one
combined_period_df = pd.concat(list_period_dfs, axis=0)
combined_period_df.reset_index(drop=True, inplace=True)
print(f"Combined period DataFrame shape: {combined_period_df.shape}")

Combined period DataFrame shape: (144, 16)


In [211]:
df_agg = add_growth(combined_period_df)
display(df_agg.sample(3))

,Report Type,Start Date,End Date,Total Cases,Unique Crime Types,Total Arrests,Total Domestic Cases,Total Violent Cases,Total Property Cases,Unique Beats,...,Unique Districts_prior,Arrest Rate_prior,Domestic Rate_prior,Violent Rate_prior,Property Rate_prior,Arrest Rate Change,Domestic Rate Change,Violent Rate Change,Property Rate Change,Case Growth %
4,R12,2022-01-31,2023-01-31,245466,31,29073,45381,34427,82006,275,...,24,12.49,22.79,15.69,25.86,-0.65,-4.30,-1.66,7.55,17.52
13,R12,2022-10-31,2023-10-31,261366,31,31046,46389,36416,89788,275,...,24,11.95,19.49,14.57,31.68,-0.07,-1.74,-0.64,2.67,11.79
9,R12,2022-06-30,2023-06-30,257640,31,29701,45492,35187,90178,275,...,24,12.07,21.21,15.47,28.40,-0.54,-3.55,-1.81,6.60,17.13


In [212]:
df_agg.to_csv(
    '../agg/chicago_crime_summary_stats.csv', index=False)
